In [1]:
import os 
import copy
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from skimage import measure

import fxsdata
from fxsdata import fxs_mouse
from at_synapse_detection import dataAccess as da
from at_synapse_detection import SynapseDetection as syn

%matplotlib notebook
#%matplotlib inline



In [5]:
def calculate_zsize_distributions(blob_vol, query_vol):
    """
    blob_vol : 3d array - raw data
    """
    blob_vol = syn.getProbMap(blob_vol)
    blob_label_vol = measure.label(blob_vol > 0.9)
    blob_stats = measure.regionprops(blob_label_vol)
    print("Number of blobs: ", len(blob_stats))
    
    query_bw = query_vol > 0.9
    query_bw = query_bw * -1 
    query_bw[query_bw==0] = 1 
    
    combinedvol = blob_label_vol * query_bw #blobs with synapses should have negative values 
    stats = measure.regionprops(blob_label_vol, combinedvol)

    z_list = [] 
    for stat in stats: 
        if stat.min_intensity < 0: 
            zsize = np.max(stat.coords[:, 2]) - np.min(stat.coords[:, 2]) + 1 
            z_list.append(zsize)
    
    print(np.unique(z_list))
    return z_list

In [6]:
mouse_list = ['1ss', '2ss', '3ss', '4ss', '5ss', '6ss', '7ss', '22ss']
mouse_suffix = '_stacks'
layer_list = ['F000', 'F001', 'F002', 'F003']
psd_suffix = '_PSD.tif'
base_fp = '/Volumes/fxs_data/yi_mice/'


In [8]:
# loop through all the filepaths 
mouse_n = 0 
allmice = {} 
for mouse_n, mouse in enumerate(mouse_list):
    print("Starting Mouse: ", mouse)
    allmice[mouse] = {} 
    query_itr = 0 
    for layer_n, layer in enumerate(layer_list): 
        print("Starting Mouse-Layer", mouse, "-", layer)
        allmice[mouse][layer] = {} 
        psd_fn = os.path.join(base_fp, mouse + mouse_suffix, layer, mouse + psd_suffix)
        print(psd_fn) 

        for q_n in range(0, 12): 
            print("Starting Mouse-Layer-query", mouse, "-", layer, "-", str(q_n))
            q_fp = os.path.join(base_fp, mouse + mouse_suffix, 'results_' \
                                + mouse + '_fragX', layer, 'query_'+str(query_itr)+'.npy')
            query_itr = query_itr + 1 
            
            psdvol = da.imreadtiff(psd_fn)
            q_vol = np.load(q_fp)
            print(q_vol.shape)
            zlist = calculate_zsize_distributions(psdvol, q_vol)
            allmice[mouse][layer][q_n] = zlist
            break 
        break 
    break 

        

Starting Mouse:  1ss
Starting Mouse-Layer 1ss - F000
/Volumes/fxs_data/yi_mice/1ss_stacks/F000/1ss_PSD.tif
Starting Mouse-Layer-query 1ss - F000 - 0
(1040, 1388, 62)
Number of blobs:  159743
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 28 29 35 43]


In [ ]:
np.save('psd_zlist.npy', allmice) 


In [ ]:
plt.figure()
plt.hist(allmice['2ss']['F000'][1], 20)

# Calculate Volume

In [9]:
mouse_list

['1ss', '2ss', '3ss', '4ss', '5ss', '6ss', '7ss', '22ss']

In [10]:
layer

'F000'

In [24]:
# loop through all the filepaths 
mouse_n = 0 
allmice_volume = {} 

res_xy_nm = 100 
res_z_nm = 70 
volume_list = [] 

for mouse_n, mouse in enumerate(mouse_list):
    allmice_volume[mouse] = {} 
    query_itr = 0 

    dapi_mask_str_base = '/Volumes/fxs_data/yi_mice/dapi-masks/' + mouse + '_stacks'
    
    for layer_n, layer in enumerate(layer_list): 

        # Load DAPI mask
        dapi_mask_fn = os.path.join(dapi_mask_str_base, layer, mouse + '-DAPI-mask.tiff')
        dapi_mask = da.imreadtiff(dapi_mask_fn)
        dapi_mask = dapi_mask.astype(np.bool)
        combined_mask = np.logical_not(dapi_mask)
        
        volume_um3 = np.count_nonzero(combined_mask) * (res_xy_nm / 1000) * (res_xy_nm / 1000) * (res_z_nm / 1000)
        print("Mouse", mouse, "-", layer, "- Volume: ", volume_um3, "um3")
        allmice_volume[mouse][layer] = volume_um3
        volume_list.append(volume_um3)

        

Mouse 1ss - F000 - Volume:  56174.346200000015 um3
Mouse 1ss - F001 - Volume:  56349.28880000001 um3
Mouse 1ss - F002 - Volume:  55471.59100000001 um3
Mouse 1ss - F003 - Volume:  55376.37630000001 um3
Mouse 2ss - F000 - Volume:  30626.999900000006 um3
Mouse 2ss - F001 - Volume:  30178.386000000006 um3
Mouse 2ss - F002 - Volume:  30033.83320000001 um3
Mouse 2ss - F003 - Volume:  33005.563500000004 um3
Mouse 3ss - F000 - Volume:  28688.378600000007 um3
Mouse 3ss - F001 - Volume:  30353.789200000007 um3
Mouse 3ss - F002 - Volume:  31827.798800000008 um3
Mouse 3ss - F003 - Volume:  34503.09590000001 um3
Mouse 4ss - F000 - Volume:  34801.90350000001 um3
Mouse 4ss - F001 - Volume:  35427.91420000001 um3
Mouse 4ss - F002 - Volume:  35358.55610000001 um3
Mouse 4ss - F003 - Volume:  37488.98720000001 um3
Mouse 5ss - F000 - Volume:  28463.549100000007 um3
Mouse 5ss - F001 - Volume:  29373.564500000004 um3
Mouse 5ss - F002 - Volume:  28295.239700000006 um3
Mouse 5ss - F003 - Volume:  30933.212800

In [29]:
np.std(volume_list)

9605.695526465004

In [30]:
np.mean(volume_list)

33897.22754687501

In [31]:
np.save('volume_dict.npy', allmice_volume) 
